<a href="https://colab.research.google.com/github/songqsh/MA2210/blob/main/src/simplex_wg_dual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplex method for the WG-dual

We are going to run simplex tableau method for the dual of the example - WG:
$$\min w = 4 y_1  + 12 y_2 + 18 y_3$$
s.t.
$$\begin{array}{lll}
y_1 & & + 3y_3 & \ge 3 \\
& 2y_2 &+2y_3 & \ge 5
\end{array}
$$
for $y_i\ge 0$.

In [15]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
import warnings
#ignore by message
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")


## Matrix in numpy

In [16]:
# input augmented matrix with big M
M = 10000000.0

AM = np.array([
               [-1, 4, 12, 18, 0, M, 0, M, 0], 
               [0, 1, 0, 3, -1, 1, 0, 0, 3], 
               [0, 0, 2, 2, 0, 0, -1, 1, 5]
               ], dtype = float)
pd.DataFrame(AM)
#print(pd.AM) # print matrix

,0,1,2,3,4,5,6,7,8
0,-1.0,4.0,12.0,18.0,0.0,10000000.0,0.0,10000000.0,0.0
1,0.0,1.0,0.0,3.0,-1.0,1.0,0.0,0.0,3.0
2,0.0,0.0,2.0,2.0,0.0,0.0,-1.0,1.0,5.0


## Pivotizing

Pivotizing with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [17]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero
    

## Simplex tableau

In [18]:
pd.DataFrame(AM) # print original augmented matrix

,0,1,2,3,4,5,6,7,8
0,-1.0,4.0,12.0,18.0,0.0,10000000.0,0.0,10000000.0,0.0
1,0.0,1.0,0.0,3.0,-1.0,1.0,0.0,0.0,3.0
2,0.0,0.0,2.0,2.0,0.0,0.0,-1.0,1.0,5.0


In [19]:
pivot(AM, 1, 5)
pd.DataFrame(AM) # print

,0,1,2,3,4,5,6,7,8
0,-1.0,-9999996.0,12.0,-29999982.0,10000000.0,0.0,0.0,10000000.0,-30000000.0
1,0.0,1.0,0.0,3.0,-1.0,1.0,0.0,0.0,3.0
2,0.0,0.0,2.0,2.0,0.0,0.0,-1.0,1.0,5.0


In [21]:
pivot(AM, 2, 7)
pd.DataFrame(AM) #print

,0,1,2,3,4,5,6,7,8
0,-1.0,-9999996.0,-19999988.0,-49999982.0,10000000.0,0.0,10000000.0,0.0,-80000000.0
1,0.0,1.0,0.0,3.0,-1.0,1.0,0.0,0.0,3.0
2,0.0,0.0,2.0,2.0,0.0,0.0,-1.0,1.0,5.0


This pivotize all pivot columns and we start this tableau with routine tableau procedure with starting BV $\{x_5, x_7\}$. Here dollar sign includes latex formula for math symbols

In [22]:
np.divide(AM[:,-1], AM[:, 3])

array([1.60, 1.00, 2.50])

In [24]:
pivot(AM, 1, 3)
pd.DataFrame(AM)

,0,1,2,3,4,5,6,7,8
0,-1.0,6.666665e+06,-19999988.0,0.0,-6.666661e+06,1.666666e+07,10000000.0,0.0,-30000018.0
1,0.0,3.333333e-01,0.0,1.0,-3.333333e-01,3.333333e-01,0.0,0.0,1.0
2,0.0,-6.666667e-01,2.0,0.0,6.666667e-01,-6.666667e-01,-1.0,1.0,3.0


In [25]:
np.divide(AM[:,-1], AM[:, 2])

array([1.50, inf, 1.50])

In [26]:
pivot(AM, 2,2)
pd.DataFrame(AM)

,0,1,2,3,4,5,6,7,8
0,-1.0,2.000000,0.0,0.0,2.000000,9.999998e+06,6.0,9999994.0,-36.0
1,0.0,0.333333,0.0,1.0,-0.333333,3.333333e-01,0.0,0.0,1.0
2,0.0,-0.333333,1.0,0.0,0.333333,-3.333333e-01,-0.5,0.5,1.5


Since the 0-row has no negative number, the last entry of the first row is the maximum value of -w, which is -36. This implies min of w is 36